![](images/pdf_to_text.png){fig-align="center"}


The ability to extract and manipulate data from PDF files is an essential skill for anyone who works with digital documents. For example: organizations, governments, universities, and businesses use themas tools to publish data. Just imagine this: I've often seen governments publish their data in pdfs as tables. As a public policy maker or as a data analyst, that can be painful; imagine translate all that information by hand or using paid tools as PDFTables or Convert from PDF to Excel online. This is where Python and the `pdfplumber` and `tabulate` libraries comes in handy: it allows us to extract text, tables, and other data from PDF files with ease, making it a valuable tool for data analysis and processing. I would also add that this skill is useful as a crucial step for Natural Language Processing since a lot of potential inputs right now are just in pdf texts. 

In the next lines, I'll show how we can extract text from PDF documents that also includes tables. This approach is easy to understand and reliable, so, anyone can posses and domain this skill; you just need a little bit of knowledge about Python and how we can work with objects.


::: callout-warning
## Important

The next approach applies only to PDF documents that have text inside and are not scanned documents. If you have a pdf document that are scanned text, then Optical character recognition (OCR) algorithms are necessary to realize this task. This method can be implemented in another entry within the Hertie Coding Club.
:::


 

If you are working within your Python environment, please install this packages. Otherwise, I'll give you the google collab where you can run Python code without installing it in your computer.

::: callout-important
## Checkout

If you are running the code locally, please be sure that you have installed the latest Java version since tabulate works within this framework. `tabulate` library uses Java to render the results.
:::


## Install packages


In [ ]:
#| eval: false

pip install pdfplumber #Library used to extract text from pdf documents
pip install pandas #Library to manipulate dataframes
pip install tabula-py tabulate #Library to extract tables from PDF documents. 

## Extract data from a PDF document


This is the [document](https://www.hertie-school.org/fileadmin/1_Study/11_Scholarships_FinancialAid/US_loans/03_MPP_Study__Examination___Admission_Rules.pdf) that we will use to this example. It's a document from the Hertie School that describes the Study, Examination, and Admission Rules in 2016 for the MPP program. Here our goal will be extract all the content from it and put it in a dataframe. 


In [ ]:
#Import libraries
import pdfplumber as pp
import pandas as pd

#Set PDF file name
pdf = 'MPP_Study_Example.pdf'

#Open PDF file and extract data from each page
with pp.open(pdf) as book:
    #Initialize an empty list to store text
    page_data = []
    #Loop over each page in the PDF file
    for page_no, page in enumerate(book.pages, start = 1):
        #Extract text from page
        data = page.extract_text()
        #Append page data to list
        page_data.append([pdf, data.strip(), page_no])

#Create pandas DataFrame from page data and set column names
df_text = pd.DataFrame(page_data, columns=['file_name', 'text', 'page_number'])

#Display DataFrame
print(df_text)


We have three columns. The first one shows the document's name where we extracted the text. The second column contains the text, and the third one the number of pages. Of course, you can modify the code and the variable names according to your necessities. 


If you want to save your dataframe, remember this functions:


In [ ]:
#| eval: false

# Save the dataframe to a CSV file
df_text.to_csv('df_text_pdf.csv', index=False)

# Save the dataframe to an Excel file
df_text.to_excel('df_text_pdf.xlsx', index=False)

If you look close, there is a table in page number 5. `PDFplumber` renders that as text and not in a table format. One approach to this would be use now the `tabulate` library to extract exclsuively tables from our document. The table inside the document is not clear or doesn't have a completely table format. However, we can still apply our function to extract that table and approach it to a nice table format. 


In [ ]:
#Import required libraries
import tabula
import numpy as np
from tabula import read_pdf
from tabulate import tabulate

#Set the path to the PDF file
pdf_path = 'MPP_Study_Example.pdf'

#Read all tables from the PDF file into a list of DataFrames
df_list = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)

#Print the list of DataFrames
print(df_list)


As you can see, we have a list with three dataframes. Our library recognized 3 tables, however, there is just really one at page 5. Now let's just keep with the one that matter to us. 


In [ ]:
print(df_list[1])

Here is our table or interest. Let's do some wrangling to clean it. 


In [ ]:
#| warning: false


#Extract the second table from the list and skip the header row
df_table = df_list[1].iloc[1:]

#Replace commas with dots and convert to numeric values
df_table["German"] = pd.to_numeric(df_table["German"].str.replace(',', '.'), 
                                                              errors='coerce')

#Rename columns
df_table = df_table.rename(columns={'German': 'Grade', 'Numerical': 'Percentage'})

#Define a function to map grades to categories. We are doing this
#since if we use a statement, our code will be larger.
def grade(score):
    if score >= 1 and score <= 1.3:
        return 'excellent'
    elif score > 1.3 and score <= 2:
        return 'very good'
    elif score > 2 and score <= 2.7:
        return 'good'
    elif score > 2.7 and score <= 3.3:
        return 'satisfactory'
    elif score > 3.3 and score < 4:
        return 'sufficient'
    elif score >= 4:
        return 'fail'
    else:
        return np.nan

#Apply the grade function to create a new column with grade categories
df_table['Grade Category'] = df_table['Grade'].apply(grade)

#Drop rows with missing grade categories
df_table = df_table.dropna(subset=['Grade Category'])

#Drop the "Definition" column
df_table = df_table.drop('Definition', axis=1)

#Extract minimum and maximum percentage values to replace the Percentage. 
#Here we use some regex to obtain just the numbers within a "-"
df_table[['Min%', 'Max%']] = df_table['Percentage'].str.extract(r'(\d+)-(\d+)%')

#Convert percentage values to float
df_table[['Min%', 'Max%']] = df_table[['Min%', 'Max%']].astype(float) / 100

#Drop the original "Percentage" column
df_table_final = df_table.drop('Percentage', axis = 1)

#Print the final table
print(df_table_final)


::: {.callout-note}
## Remember

To save a dataframe or your table, use this functions:


In [ ]:
#| eval: false

# Save the dataframe to a CSV file
df_table_final.to_csv('df_text_pdf.csv', index=False)

# Save the dataframe to an Excel file
df_table_final.to_excel('df_text_pdf.xlsx', index=False)

:::


 

## Extract tables from a PDF document


As you can see, the last example was just a form to deal with a tabl when we don't have a clear pattern of it in a document. Generally, we need another tool to extract a table from a document since it will be much easier because of the `tabula` library it's trained just to recognize tables. Now, we will use another document with cleaner tables, and the library can work even better. We will use a [document](https://sedl.org/afterschool/toolkits/science/pdf/ast_sci_data_tables_sample.pdf) that contains text but also much cleaner tables. 


In [ ]:
import tabula
from tabula import read_pdf
from tabulate import tabulate

df_list_example2 = tabula.read_pdf('ast_sci_data_tables_sample.pdf', 
                                    pages="all", multiple_tables = True)

#We use len to determine how many tables the document has
len(df_list_example2)

We have for tables. Let's see them closer:


In [ ]:
print(df_list_example2)

Looks good! Let's say that I just want the second table, so, I use the index of the list to keep with my dataframe.


In [ ]:
print(df_list_example2[1])

# Conclusion 

------------------------------------------------------------------------

Now, we can use this for our own purposes. Asou can see, the process was much straight forward. In conclusion, this is just one approach of multiple ones since there are different libraries depending of what you want to do. However, I like this libraries since are easy to understand. Finally, if you want to reproduce this code, here I leave a Google Colab notebook for you to input your pdf's and get the text out of it. This step is super interesting and I would say necessary to start a pipeline for NLP models, analysis of text, etc. 


 

<div style="text-align:center;">
[<img src="https://colab.research.google.com/assets/colab-badge.svg" width="250">](https://colab.research.google.com/drive/1qBVCIpBReACYi1Pi81bOWZNw66XsY2mA?usp=sharing)
</div>

 
 


------------------------------------------------------------------------




::: callout-note
## Reference
Cite this page: Roa, J. (2023, April 18). *PDF to text*. Hertie Coding Club. [URL](https://www.hertiecodingclub.com/learn/python/pdf_to_text/)
:::
